# Тестовое задание
## SQL

### 2.1 Очень усердные ученики.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".
Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Дана таблица **default.peas**

|Название атрибута|Тип атрибута|Смысловое значение|
|-----------|-----------|-----------|
|st_id|int|ID ученика|
|timest|timestamp|Время решения карточки|
|correct|bool|Правильно ли решена горошина?|
|subject|text|Дисциплина, в которой находится горошина|

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов

Отправлять запросы буду из JupyterHub с помощью библиотеки pandahouse

In [1]:
from CH import Getch

In [2]:
# формирую SQL-запрос
q1 = '''
-- объявляю параметр с текущим месяцем
WITH
    (SELECT toMonth(MAX(timest)) FROM {db}.peas) AS current_month

SELECT countIf(peas_count>=20) as diligent_students
FROM
    (SELECT
        st_id,
        countIf(correct=1 and toMonth(timest)=current_month) AS peas_count -- количество решённых горошин в текущем месяце
    FROM {db}.peas
    GROUP BY st_id)'''

In [3]:
# отправляю запрос и записываю результат в датафрейм
answer1 = Getch(q1)
answer1.df

,diligent_students
0,136


### 2.2 Оптимизация воронки

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Даны таблицы: **default.peas** (см. выше), **default.studs**:

|Название атрибута|Тип атрибута|Смысловое значение|
|-----------|-----------|-----------|
|st_id|int|ID ученика|
|test_grp|text|Метка ученика в данном эксперименте|

и **default.final_project_check**:

|Название атрибута|Тип атрибута|Смысловое значение|
|-----------|-----------|-----------|
|st_id|int|ID ученика|
|sale_time|timestamp|Время покупки|
|money|int|Цена, по которой приобрели данный курс|
|subject|text |Дисциплина, в которой был приобретён курс|

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

**Активным** считается пользователь, за все время решивший **больше** 10 задач правильно в любых дисциплинах.

**Активным по математике** считается пользователь, за все время решивший **2 или больше задач** правильно по математике.

В процессе написания запроса выяснил, что в таблицах для задания параметр **join_use_nulls**, который устанавливает тип поведения **JOIN**, равен **0**. Это значит, что пустые ячейки, возникающие при отсутствии соответствия между таблицами по ключу, заполняются значением по умолчанию соответствующего типа. В данном задании пустые ячейки с типом поля Int будут заполняться нулями, что поможет корректно посчитать **ARPU** (среднее значение выручки по пользователю), и сравнение с нулём будет условием для подсчёта количества строк.

За основу взял табличку **default.studs** с 600 пользователями разбитыми на группы. С помощью **LEFT JOIN** подтянул доходы по пользователю и количество решённых задач. На основе получившейся структуры рассчитал метрики по двум группам.  

In [4]:
# формирую SQL-запрос
q2 = '''
WITH
    solved_problems AS -- табличка с количеством решённых задач по пользователю + группа пользователя
        (SELECT 
            st_id,
            test_grp,
            countIf(correct=1) as solved_problems,
            countIf(correct=1 and subject='Math') as solved_math_problems
        FROM {db}.studs
        LEFT JOIN {db}.peas USING(st_id)
        GROUP BY st_id, test_grp),
        
    revenue_by_users AS -- табличка с доходом по пользователю
        (SELECT
            st_id,
            sum(money) as revenue,
            sumIf(money, subject='Math') as math_revenue
        FROM {db}.final_project_check
        GROUP BY st_id)


SELECT -- основная часть запроса с расчётом метрик
    test_grp,
    avg(revenue) as ARPU,
    sum(revenue) / countIf(solved_problems>10) as ARPAU,
    countIf(revenue>0) / count(st_id) as CR,
    countIf(revenue>0) / countIf(solved_problems>10) as CR_active,
    countIf(math_revenue>0) /
    countIf(solved_math_problems>=2) as CR_math
FROM solved_problems
LEFT JOIN revenue_by_users USING(st_id)
GROUP BY test_grp'''

In [5]:
# отправляю запрос и записываю результат в датафрейм
answer2 = Getch(q2)
answer2.df.round(2)

,test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4540.98,10905.51,0.05,0.12,0.16
1,pilot,11508.47,35364.58,0.11,0.33,0.38


В экспериментальной группе виден рост всех метрик :)